In [1]:
#0: imports

import uproot 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from imports.data import CCV3
from torch_geometric.data import DataLoader 
from imports.models import Net_DE, Net_GAT, Net_Trans
from torch_geometric.nn import knn_graph

import numpy as np
import awkward as ak
import time
from imports.Agglomerative import Aggloremative

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [2]:
Ppath = '/vols/cms/mm1221/Data/100k/5pi/test/'
Epath = '/vols/cms/mm1221/Data/100k/5e/test/'
# Load test data
data_P = CCV3(Ppath, max_events=500)
P_loader = DataLoader(data_P, batch_size=1, shuffle=False, follow_batch=['x'])

data_E = CCV3(Epath, max_events=500)
E_loader = DataLoader(data_E, batch_size=1, shuffle=False, follow_batch=['x'])


### Loading data


  0%|                                                                                                                 | 0/1 [00:40<?, ?it/s]
/cvmfs/sft.cern.ch/lcg/views/LCG_105a_cuda/x86_64-el9-gcc11-opt/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Reached 500!
### Loading data


  0%|                                                                                                                 | 0/1 [00:37<?, ?it/s]

Reached 500!


In [3]:
import uproot

# Define the file paths
data_Ep_path = '/vols/cms/mm1221/Data/100k/5e/test/raw/test.root'  # electron file
data_Pp_path = '/vols/cms/mm1221/Data/100k/5pi/test/raw/test.root'  # pion file

# Open the ROOT files using uproot
file_Ep = uproot.open(data_Ep_path)
file_Pp = uproot.open(data_Pp_path)

# Load the branches for electrons (data_Ep)
GT_ind_Ep   = file_Ep['simtrackstersCP']['vertices_indexes'].array()
GT_mult_Ep  = file_Ep['simtrackstersCP']['vertices_multiplicity'].array()
energies_Ep = file_Ep['clusters']['energy'].array()
LC_x_Ep     = file_Ep['clusters']['position_x'].array()

# Load the branches for pions (data_Pp)
GT_ind_Pp   = file_Pp['simtrackstersCP']['vertices_indexes'].array()
GT_mult_Pp  = file_Pp['simtrackstersCP']['vertices_multiplicity'].array()
energies_Pp = file_Pp['clusters']['energy'].array()

LC_x_Pp     = file_Ep['clusters']['position_x'].array()

# Create a skim mask to filter out events with 0 calorimeter particles (for electrons)
skim_mask_Ep = [len(e) >= 1 for e in LC_x_Ep]

# Apply the skim mask to filter the electron arrays
GT_ind_Ep   = GT_ind_Ep[skim_mask_Ep]
GT_mult_Ep  = GT_mult_Ep[skim_mask_Ep]
energies_Ep = energies_Ep[skim_mask_Ep]


# Create a skim mask for pions
skim_mask_Pp = [len(e) >= 1 for e in LC_x_Pp]

# Apply the skim mask to filter the pion arrays
GT_ind_Pp   = GT_ind_Pp[skim_mask_Pp]
GT_mult_Pp  = GT_mult_Pp[skim_mask_Pp]
energies_Pp = energies_Pp[skim_mask_Pp]

In [4]:
import awkward as ak

# ----- For pions -----


# Initialize lists to store filtered results for pions
filtered_GT_ind_P = []
filtered_GT_mult_P = []

# Loop over events for pions
for event_idx, track_indices in enumerate(data_P.stsCP_vertices_indexes):
    # Flatten the current event's track indices and convert to a set
    track_flat = set(ak.flatten(track_indices).tolist())
    
    # Get the current event's GT arrays
    event_GT_ind = GT_ind_Pp[event_idx]
    event_GT_mult = GT_mult_Pp[event_idx]
    
    # Initialize lists for the filtered sublists in the current event
    filtered_event_GT_ind = []
    filtered_event_GT_mult = []
    
    # Loop over the sublists and filter using the track_flat set
    for sublist_ind, sublist_mult in zip(event_GT_ind, event_GT_mult):
        filtered_sublist_ind = [idx for idx in sublist_ind if idx in track_flat]
        filtered_sublist_mult = [mult for idx, mult in zip(sublist_ind, sublist_mult) if idx in track_flat]
        filtered_event_GT_ind.append(filtered_sublist_ind)
        filtered_event_GT_mult.append(filtered_sublist_mult)
    
    # Append the filtered event arrays to the output lists for pions
    filtered_GT_ind_P.append(filtered_event_GT_ind)
    filtered_GT_mult_P.append(filtered_event_GT_mult)

# Convert the filtered results back into awkward Arrays for pions
GT_ind_filt_P = ak.Array(filtered_GT_ind_P)
GT_mult_filt_P = ak.Array(filtered_GT_mult_P)

# ----- For electrons -----

# Initialize lists to store filtered results for electrons
filtered_GT_ind_E = []
filtered_GT_mult_E = []

# Loop over events for electrons
for event_idx, track_indices in enumerate(data_E.stsCP_vertices_indexes):

    # Flatten the current event's track indices and convert to a set
    track_flat = set(ak.flatten(track_indices).tolist())
    
    # Get the current event's GT arrays
    event_GT_ind = GT_ind_Ep[event_idx]
    event_GT_mult = GT_mult_Ep[event_idx]
    
    # Initialize lists for the filtered sublists in the current event
    filtered_event_GT_ind = []
    filtered_event_GT_mult = []
    
    # Loop over the sublists and filter using the track_flat set
    for sublist_ind, sublist_mult in zip(event_GT_ind, event_GT_mult):
        filtered_sublist_ind = [idx for idx in sublist_ind if idx in track_flat]
        filtered_sublist_mult = [mult for idx, mult in zip(sublist_ind, sublist_mult) if idx in track_flat]
        filtered_event_GT_ind.append(filtered_sublist_ind)
        filtered_event_GT_mult.append(filtered_sublist_mult)
    
    # Append the filtered event arrays to the output lists for electrons
    filtered_GT_ind_E.append(filtered_event_GT_ind)
    filtered_GT_mult_E.append(filtered_event_GT_mult)

# Convert the filtered results back into awkward Arrays for electrons
GT_ind_filt_E = ak.Array(filtered_GT_ind_E)
GT_mult_filt_E = ak.Array(filtered_GT_mult_E)


In [7]:
model = Net_Trans(128,3, dropout=0.3, contrastive_dim=16, num_heads=4)
checkpoint= torch.load('/vols/cms/mm1221/hgcal/Mixed/LC/Full/runs/Trans/hd128nl3cd16nh4k32/best_model.pt',  map_location=torch.device('cpu'))
#checkpoint= torch.load('/vols/cms/er421/hgcal/code/code/Mixed/LC/Full/results/hd128nl3cd16k64/epoch-100.pt',  map_location=torch.device('cpu'))
model.load_state_dict(checkpoint)  
model.eval()  


Net_Trans(
  (lc_encode): Sequential(
    (0): Linear(in_features=8, out_features=128, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ELU(alpha=1.0)
  )
  (transformer_layers): ModuleList(
    (0-2): 3 x TransformerConv(128, 128, heads=4)
  )
  (output): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ELU(alpha=1.0)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): ELU(alpha=1.0)
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=32, out_features=16, bias=True)
  )
)

In [8]:
import time
import numpy as np
import matplotlib.pyplot as plt

def calculate_metrics(df, model_name):
    # ----- Efficiency Calculation -----
    # Step 1: Filter out rows where 'cp_id' is NaN
    cp_valid = df.dropna(subset=['cp_id']).copy()

    # Step 2: Group by 'event_index' and 'cp_id' to proess each CaloParticle individually
    cp_grouped = cp_valid.groupby(['event_index', 'cp_id'])

    # Step 3: For each CaloParticle, check if any 'shared_energy' >= 50% of 'cp_raw_energy'
    def is_cp_associated(group):
        cp_raw_energy = group['cp_energy'].iloc[0]  # Assuming 'cp_raw_energy' is consistent within the group
        threshold = 0.8 * cp_raw_energy
        return (group['shared_energy'] >= threshold).any()

    # Apply the association function to each group
    cp_associated = cp_grouped.apply(is_cp_associated)

    # Step 4: Calculate the number of associated CaloParticles and total CaloParticles
    num_associated_cp = cp_associated.sum()
    total_cp = cp_associated.count()
    efficiency = num_associated_cp / total_cp if total_cp > 0 else 0

    # ----- Purity Calculation -----
    tst_valid = df.dropna(subset=['trackster_id']).copy()
    tst_grouped = tst_valid.groupby(['event_index', 'trackster_id'])
    tst_associated = tst_grouped['reco_to_sim_score'].min() < 0.2
    num_associated_tst = tst_associated.sum()
    total_tst = tst_associated.count()
    purity = num_associated_tst / total_tst if total_tst > 0 else 0
    
    # ----- Ratio between num of tracksters to caloparticles Calculation -----
    num_tracksters_ratio = total_tst / total_cp if total_cp > 0 else 0
    
    # Print results for the model
    print(f"\nModel: {model_name}")
    print(f"Efficiency: {efficiency:.4f} ({num_associated_cp} associated CPs out of {total_cp} total CPs)")
    print(f"FR: {1 - purity:.4f} ({num_associated_tst} associated Tracksters out of {total_tst} total Tracksters)")
    print(f"Num tracksters ratio: {num_tracksters_ratio}")

    return {
        'efficiency': efficiency,
        'purity': purity,
        'Num_tracksters_ratio': num_tracksters_ratio,
    }

In [9]:
from tqdm import tqdm
def calculate_reco_to_sim_score_and_sharedE(ReconstructedTrackster, energies_indices, CaloParticle, calo_mult):
    """
    Calculate the reco-to-sim score for a given ReconstructedTrackster and CaloParticle.

    Parameters:
    - ReconstructedTrackster: array of DetIds in the ReconstructedTrackster.
    - energies_indices: array of energies associated with all DetIds (indexed by DetId).
    - CaloParticle: array of DetIds in the CaloParticle.

    Returns:
    - reco_to_sim_score: the calculated reco-to-sim score.
    """
    numerator = 0.0
    denominator = 0.0
    sharedEnergy = 0.0

    # Iterate over all DetIds in the ReconstructedTrackster
    for i, det_id in enumerate(ReconstructedTrackster):
        energy_k = energies_indices[det_id]  # Energy for the current DetId in the Trackster
        
        # Fraction of energy in the Trackster (fr_k^TST)
        fr_tst_k = 1 

        #Fraction of energy in the caloparticle
        if det_id in CaloParticle:
            index = np.where(CaloParticle == det_id)[0][0]
            fr_sc_k = 1 / calo_mult[index]
            
        else:
            fr_sc_k = 0 # binary function also for CaloParticle
            
        # Update numerator using the min function
        numerator += min(
            (fr_tst_k - fr_sc_k) ** 2,  # First term in the min function
            fr_tst_k ** 2               # Second term in the min function
        ) * (energy_k ** 2)

        # Update denominator
        denominator += (fr_tst_k ** 2) * (energy_k ** 2)
        
        #shared_energy calculation
        recosharedEnergy = energy_k * fr_tst_k
        simsharedEnergy = energy_k * fr_sc_k
        sharedEnergy += min(simsharedEnergy,recosharedEnergy)
        
        

    # Calculate score
    reco_to_sim_score = numerator / denominator if denominator != 0 else 1.0
    return reco_to_sim_score, sharedEnergy


def calculate_all_event_scores(GT_ind, GT_mult, energies, recon_ind,  num_events = 100):
    """
    Calculate sim-to-reco and reco-to-sim scores for all CaloParticle and ReconstructedTrackster combinations across all events.

    Parameters:
    - GT_ind: List of CaloParticle indices for all events.
    - energies: List of energy arrays for all events.
    - recon_ind: List of ReconstructedTrackster indices for all events.
    - LC_x, LC_y, LC_z, LC_eta: Lists of x, y, z positions and eta values for all DetIds across events.

    Returns:
    - DataFrame containing scores and additional features for each CaloParticle-Trackster combination across all events.
    """
    # Initialize an empty list to store results
    all_results = []

    # Loop over all events with a progress bar
    for event_index in tqdm(range(num_events)):
        caloparticles = GT_ind[event_index]  # Indices for all CaloParticles in the event
        tracksters = recon_ind[event_index]  # Indices for all ReconstructedTracksters in the event
        event_energies = energies[event_index]  # Energies for this event
        event_GT_mult = GT_mult[event_index]
        
        # Loop over all CaloParticles
        for calo_idx, caloparticle in enumerate(caloparticles):
            calo_mult = event_GT_mult[calo_idx]
            cp_raw_energy_lc = event_energies[caloparticle] / calo_mult
            cp_raw_energy = np.sum(cp_raw_energy_lc)

            
            for trackster_idx, trackster in enumerate(tracksters):

                
                # Calculate reco-to-sim score
                reco_to_sim_score, shared_energy = calculate_reco_to_sim_score_and_sharedE(trackster, event_energies, caloparticle,calo_mult)
                # Calculate trackster energy

                # Append results
                all_results.append({
                    "event_index": event_index,
                    "cp_id": calo_idx,
                    "trackster_id": trackster_idx,
                    "reco_to_sim_score": reco_to_sim_score,
                    "cp_energy": cp_raw_energy,
                    "shared_energy": shared_energy,
                })

    # Convert results to a DataFrame
    df = pd.DataFrame(all_results)
    return df

In [16]:
print(all_predictions_E)

[array([[-0.48390728,  0.6888035 ,  2.9245234 , ..., -0.3965365 ,
         -0.5602754 ,  0.38261485],
        [-0.50836223,  0.7206374 ,  2.9306953 , ..., -0.39806604,
         -0.5410636 ,  0.33663154],
        [-0.5090022 ,  0.71020633,  2.9255297 , ..., -0.38669363,
         -0.55619556,  0.37010372],
        ...,
        [-0.24173701,  1.202589  ,  1.8986366 , ..., -0.08121085,
          0.30100402,  1.7933283 ],
        [-0.2446779 ,  1.1484604 ,  1.358118  , ..., -0.2442919 ,
          0.5100504 ,  2.3845043 ],
        [-0.21707982,  1.4190519 ,  1.4469141 , ..., -0.06537735,
          0.4356977 ,  2.3535213 ]], dtype=float32)
 array([[ 2.479198  , -0.26253682,  0.00967902, ...,  0.82334757,
          1.0457658 ,  1.5686591 ],
        [ 2.537668  , -0.24375302,  0.03156132, ...,  0.82706296,
          1.0011325 ,  1.5174779 ],
        [ 2.378924  , -0.14610332,  0.12100738, ...,  0.8115994 ,
          0.96888036,  1.4472898 ],
        ...,
        [ 2.644172  , -0.3028409 ,  0.03

In [19]:
import numpy as np
import awkward as ak
import pandas as pd

# ---------------------------------------------------------
# 1) Define your threshold scan with more data points:
#    - From 0 to 0.4: every 0.005
#    - From 0.4 to 1: every 0.02
# ---------------------------------------------------------
first_segment = np.arange(0, 0.5, 0.01)
second_segment = np.arange(0.5, 1.0, 0.05)  # 1.02 to ensure inclusion of 1.0
threshold_values = np.concatenate((first_segment, second_segment))

# ---------------------------------------------------------
# 2) Obtain your predictions from the trained model 
#    for electrons and pions separately
# ---------------------------------------------------------
all_predictions_E = []

for i, data in enumerate(E_loader):
    if i > 50:
        break
    edge_index = knn_graph(data.x[:, :3], k=32, batch=data.x_batch)
    
    # "pred" is typically (batch_size, 16)
    pred, _ = model(data.x, edge_index, data.x_batch)
    pred = pred.detach().cpu().numpy()  # shape (N,16)
    
    # Keep appending each batch’s predictions
    all_predictions_E.append(pred)





# ---------------------------------------------------------
# 3) Prepare arrays for storing results for each dataset
# ---------------------------------------------------------
efficiencies_E, fakerates_E, ratio_E = [], [], []


# ---------------------------------------------------------
# 4) Loop over thresholds and compute metrics for electrons
# ---------------------------------------------------------
for t in threshold_values:
    print(f"\n[Electrons] Threshold = {t:.3f}")
    
    # --- 4.1 Cluster using your Aggloremative function (or other) ---
    all_cluster_labels_E = Aggloremative(all_predictions_E, threshold=t)
    
    # --- 4.2 Reconstruct tracksters ---
    # Build "recon_ind_E" by grouping cluster indices for each event
    recon_ind_E = []
    for event_idx, labels in enumerate(all_cluster_labels_E):
        event_clusters = {}
        for cluster_idx, cluster_label in enumerate(labels):
            if cluster_label not in event_clusters:
                event_clusters[cluster_label] = []
            # Use the flattened electron index arrays
            event_clusters[cluster_label].append(
                ak.flatten(data_E.stsCP_vertices_indexes[event_idx])[cluster_idx]
            )
        # Convert the dict of lists into a list-of-lists
        recon_ind_E.append([event_clusters[label] for label in sorted(event_clusters.keys())])
    recon_ind_E = ak.Array(recon_ind_E)
    
    # --- 4.3 Calculate event scores ---
    # Use your *filtered* GT arrays and energies for electrons
    df_CL_temp_E = calculate_all_event_scores(
        GT_ind_filt_E, 
        GT_mult_filt_E, 
        energies_Ep, 
        recon_ind_E, 
        num_events=200  # using 50 events
    )
    
    # --- 4.4 Compute metrics and store in lists ---
    metrics_E = calculate_metrics(df_CL_temp_E, f"Threshold {t:.3f}")
    efficiencies_E.append(metrics_E['efficiency'])
    fakerates_E.append(1.0 - metrics_E['purity'])  
    ratio_E.append(metrics_E['Num_tracksters_ratio'])
    
df_electrons = pd.DataFrame({
    'threshold': threshold_values,
    'efficiency': efficiencies_E,
    'fake_rate': fakerates_E,
    'num_tracksters_ratio': ratio_E,
})


# ---------------------------------------------------------
# 7) (Optional) Display or save the DataFrames
# ---------------------------------------------------------
print("\n=== Electron Results ===")
print(df_electrons)

# Optionally, you can save these DataFrames to CSV files:
# df_electrons.to_csv("electron_scan.csv", index=False)
# df_pions.to_csv("pion_scan.csv", index=False)



[Electrons] Threshold = 0.000


 26%|██████████████████████████                                                                            | 51/200 [00:15<00:44,  3.37it/s]


ValueError: in ListOffsetArray64 attempting to get 51, index out of range

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.3/src/libawkward/array/ListOffsetArray.cpp#L682)

In [90]:
all_predictions_P = []
for i, data in enumerate(P_loader):  # P_loader => DataLoader for pion data
    pred = model(data.x, data.x_batch)
    all_predictions_P.append(pred[0].detach().cpu().numpy())
all_predictions_P = np.array(all_predictions_P)
efficiencies_P, fakerates_P, ratio_P = [], [], []
# ---------------------------------------------------------
# 5) Loop over thresholds and compute metrics for pions
# ---------------------------------------------------------
for t in threshold_values:
    print(f"\n[Pions] Threshold = {t:.3f}")
    
    # --- 5.1 Cluster using your Aggloremative function (or other) ---
    all_cluster_labels_P = Aggloremative(all_predictions_P, threshold=t)
    
    # --- 5.2 Reconstruct tracksters ---
    recon_ind_P = []
    for event_idx, labels in enumerate(all_cluster_labels_P):
        event_clusters = {}
        for cluster_idx, cluster_label in enumerate(labels):
            if cluster_label not in event_clusters:
                event_clusters[cluster_label] = []
            # Use the flattened pion index arrays
            event_clusters[cluster_label].append(
                ak.flatten(data_P.stsCP_vertices_indexes[event_idx])[cluster_idx]
            )
        # Convert the dict of lists into a list-of-lists
        recon_ind_P.append([event_clusters[label] for label in sorted(event_clusters.keys())])
    recon_ind_P = ak.Array(recon_ind_P)
    
    # --- 5.3 Calculate event scores ---
    # Use your *filtered* GT arrays and energies for pions
    df_CL_temp_P = calculate_all_event_scores(
        GT_ind_filt_P, 
        GT_mult_filt_P, 
        energies_Pp, 
        recon_ind_P, 
        num_events=50  # using 50 events
    )
    
    # --- 5.4 Compute metrics and store in lists ---
    metrics_P = calculate_metrics(df_CL_temp_P, f"Threshold {t:.3f}")
    efficiencies_P.append(metrics_P['efficiency'])
    fakerates_P.append(1.0 - metrics_P['purity'])  
    ratio_P.append(metrics_P['Num_tracksters_ratio'])

# ---------------------------------------------------------
# 6) Build separate DataFrames for electrons and pions
# ---------------------------------------------------------


df_pions = pd.DataFrame({
    'threshold': threshold_values,
    'efficiency': efficiencies_P,
    'fake_rate': fakerates_P,
    'num_tracksters_ratio': ratio_P,
})


[Pions] Threshold = 0.000


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:09<00:00,  5.29it/s]



Model: Threshold 0.000
Efficiency: 0.0141 (2 associated CPs out of 142 total CPs)
FR: 0.0021 (9707 associated Tracksters out of 9727 total Tracksters)
Num tracksters ratio: 68.5

[Pions] Threshold = 0.005


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:07<00:00,  6.34it/s]



Model: Threshold 0.005
Efficiency: 0.4085 (58 associated CPs out of 142 total CPs)
FR: 0.0305 (2133 associated Tracksters out of 2200 total Tracksters)
Num tracksters ratio: 15.492957746478874

[Pions] Threshold = 0.010


...


Model: Threshold 0.960
Efficiency: 1.0000 (142 associated CPs out of 142 total CPs)
FR: 0.5200 (24 associated Tracksters out of 50 total Tracksters)
Num tracksters ratio: 0.352112676056338

[Pions] Threshold = 0.980


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:07<00:00,  6.99it/s]



Model: Threshold 0.980
Efficiency: 1.0000 (142 associated CPs out of 142 total CPs)
FR: 0.5200 (24 associated Tracksters out of 50 total Tracksters)
Num tracksters ratio: 0.352112676056338

[Pions] Threshold = 1.000


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:07<00:00,  7.02it/s]


Model: Threshold 1.000
Efficiency: 1.0000 (142 associated CPs out of 142 total CPs)
FR: 0.5200 (24 associated Tracksters out of 50 total Tracksters)
Num tracksters ratio: 0.352112676056338


In [75]:
print(len(GT_ind_filt_P[34]))
print(len(GT_mult_filt_P[34]))

2
2


In [57]:
df_electrons = pd.DataFrame({
    'threshold': threshold_values,
    'efficiency': efficiencies_E,
    'fake_rate': fakerates_E,
    'num_tracksters_ratio': ratio_E,
})

In [2]:
import pandas as pd
import os

# Path to folder
csv_folder = "csv_Agg"

# Loop through all CSV files in the folder
for filename in os.listdir(csv_folder):
    if filename.endswith(".csv"):
        df_name = filename.replace(".csv", "")  # Strip .csv to use as variable name
        file_path = os.path.join(csv_folder, filename)
        globals()[df_name] = pd.read_csv(file_path)  # Create variable in global scope
        print(f"Loaded {df_name}")


Loaded electron_GAT_hard
Loaded electron_GAT
Loaded pion_Trans_hard
Loaded electron_Trans
Loaded electron_Trans_hard
Loaded pion_DE
Loaded pion_GAT
Loaded electron_DE
Loaded pion_Trans
Loaded pion_GAT_hard
Loaded electron_DE_Hard
Loaded pion_DE_Hard


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import csv

# Model types and display names
model_types = ["GAT", "DE", "Trans"]
model_titles = {
    "GAT": "Graph Attention",
    "DE": "DynamicEdgeConv",
    "Trans": "Transformer"
}

# Mapping of curve names to actual DataFrame variables
df_name_map = {
    "electron_GAT": "electron_GAT",
    "electron_GAT_hard": "electron_GAT_hard",
    "pion_GAT": "pion_GAT",
    "pion_GAT_hard": "pion_GAT_hard",

    "electron_DE": "electron_DE",
    "electron_DE_hard": "electron_DE_Hard",
    "pion_DE": "pion_DE",
    "pion_DE_hard": "pion_DE_Hard",

    "electron_Trans": "electron_Trans",
    "electron_Trans_hard": "electron_Trans_hard",
    "pion_Trans": "pion_Trans",
    "pion_Trans_hard": "pion_Trans_hard",
}

# Colors for particles
base_colors = {
    "electron": "blue",
    "pion": "red"
}

# Store all AUCs
auc_scores = {}

for model_type in model_types:
    title = model_titles[model_type]
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_title(title, fontsize=18)
    
    for particle in ["electron", "pion"]:
        for hardness in ["", "_hard"]:
            key = f"{particle}_{model_type}{hardness}"
            df_var = df_name_map.get(key)
            if df_var not in globals():
                print(f"⚠️ DataFrame '{df_var}' not found. Skipping.")
                continue

            df = globals()[df_var].sort_values("threshold").reset_index(drop=True)

            fake_rate = df["fake_rate"].values
            efficiency = df["efficiency"].values
            threshold = df["threshold"].values
            ratio = df["num_tracksters_ratio"].values

            # Pad endpoints
            if fake_rate[0] > 0:
                fake_rate = np.insert(fake_rate, 0, 0.0)
                efficiency = np.insert(efficiency, 0, 0.0)
            if fake_rate[-1] < 1:
                fake_rate = np.append(fake_rate, 1.0)
                efficiency = np.append(efficiency, 1.0)

            # Compute AUC
            auc = np.trapz(efficiency, fake_rate)
            auc_scores[key] = auc

            # Find optimal threshold
            distances = np.sqrt((fake_rate - 0)**2 + (efficiency - 1)**2)
            if len(ratio) != len(distances):
                pad = len(distances) - len(ratio)
                ratio = np.insert(ratio, 0, [1.0] * pad)
            combined = distances + np.abs(ratio - 1)
            idx = np.argmin(combined)
            opt_T = threshold[idx] if idx < len(threshold) else threshold[-1]

            # Plot
            color = base_colors[particle]
            ls = '--' if hardness else '-'
            label = (
                f"{particle.capitalize()}{' Hard' if hardness else ''} "
                f"(AUC={auc:.3f}, T={opt_T:.2f})"
            )
            ax.plot(fake_rate, efficiency, ls, color=color, label=label)

    ax.set_xlabel("Fake Rate", fontsize=14)
    ax.set_ylabel("Efficiency", fontsize=14)
    ax.tick_params(labelsize=12)
    ax.grid(True)
    ax.legend(fontsize=12)

    # Save and close
    filename = f"{title}.pdf"            # e.g. "Graph Attention.pdf"
    fig.savefig(filename, bbox_inches='tight')
    plt.close(fig)
    print(f"✅ Saved '{filename}'")

# (Optional) write out AUCs
with open("auc_scores.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Curve", "AUC"])
    for name, auc in sorted(auc_scores.items()):
        writer.writerow([name, f"{auc:.6f}"])
print("✅ All AUCs saved to 'auc_scores.csv'")


✅ Saved 'Graph Attention.pdf'
✅ Saved 'DynamicEdgeConv.pdf'
✅ Saved 'Transformer.pdf'
✅ All AUCs saved to 'auc_scores.csv'
